# Lab 1: Multiple Linear Regression

Data : https://archive.ics.uci.edu/ml/datasets/Energy+efficiency

Analyze this dataset and conclude the influence of the following factors on the heating and cooling loads of a building:
* Relative Compactness
* Surface Area
* Wall Area
* Roof Area
* Overall Height (low or high)
* Orientation  (2=north, 3=east, 4=south, 5=west)
* Glazing Area
* Glazing Area Distribution (0 = no glazing, 1= uniform, 2=north, 3=east, 4=south, 5=west)

Make powerpoint slides of your findings explaining:
* Exploratory data analysis (EDA) and data cleaning process
* The model: what do you use? what are the predictors? 
* How good is the model? Need to compared to the model with no predictors. 
* The interpretation of the model (don't forget to check for multicolinearity)


In [2]:
import pandas as pd

In [3]:
!pip install openpyxl

In [30]:
d = pd.read_excel('data/ENB2012_data.xlsx')

In [31]:
d

,relative_area,surface_area,wall_area,roof_area,height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.50,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.50,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.50,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.50,3.5,4,0.4,5,16.48,16.61


In [32]:
d.describe()

,relative_area,surface_area,wall_area,roof_area,height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


In [33]:
for col in d.columns:
    print (col)
    print (d[col].value_counts())

relative_area
0.98    64
0.90    64
0.86    64
0.82    64
0.79    64
0.76    64
0.74    64
0.71    64
0.69    64
0.66    64
0.64    64
0.62    64
Name: relative_area, dtype: int64
surface_area
514.5    64
563.5    64
588.0    64
612.5    64
637.0    64
661.5    64
686.0    64
710.5    64
735.0    64
759.5    64
784.0    64
808.5    64
Name: surface_area, dtype: int64
wall_area
294.0    192
318.5    192
343.0    128
416.5     64
245.0     64
269.5     64
367.5     64
Name: wall_area, dtype: int64
roof_area
220.50    384
147.00    192
122.50    128
110.25     64
Name: roof_area, dtype: int64
height
7.0    384
3.5    384
Name: height, dtype: int64
orientation
2    192
3    192
4    192
5    192
Name: orientation, dtype: int64
glazing_area
0.10    240
0.25    240
0.40    240
0.00     48
Name: glazing_area, dtype: int64
glazing_area_distribution
1    144
2    144
3    144
4    144
5    144
0     48
Name: glazing_area_distribution, dtype: int64
heating_load
15.16    6
13.00    5
15.23    4
2


Three categorical variables
1) height (low and high)
2) distribution (uniform, north, east, south, west)
3) orientation (north, east, south, west)

In [34]:
d['orientation'] = d['orientation'].replace({2:'north', 3:'east', 4:'south', 5:'west'})
d['glazing_area_distribution'] = d['glazing_area_distribution'].replace({0:'no glazing', 1:'uniform',2:'north', 3:'east', 4:'south', 5:'west'})
d['height'] = d['height'].apply(lambda x: 'low' if x == 3.5 else 'high')

In [35]:
!pip install patsy

In [36]:
from sklearn.linear_model import LinearRegression
from patsy import dmatrices

features = d.columns.to_list()
features.remove('cooling_load')
features.remove('heating_load')
features_str = '+'.join(features)
y, X = dmatrices("cooling_load ~ {}".format(features_str), data=d, return_type='dataframe')
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

LinearRegression(fit_intercept=False)

In [37]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
prediction = model.predict(X)
print(np.sqrt(mean_squared_error(y, prediction)))
print(r2_score(y,prediction))

3.147233149779972
0.8904125560180348


In [38]:
pd.DataFrame({'predictors':X.columns, 'coefficients': model.coef_[0].round(4)})

,predictors,coefficients
0,Intercept,127.7050
1,height[T.low],-14.9935
2,orientation[T.north],0.2920
3,orientation[T.south],0.1678
4,orientation[T.west],0.6411
5,glazing_area_distribution[T.no glazing],-1.6400
6,glazing_area_distribution[T.north],0.3374
7,glazing_area_distribution[T.south],0.3557
8,glazing_area_distribution[T.uniform],0.5201
9,glazing_area_distribution[T.west],0.0556


In [40]:
d[['relative_area', 'surface_area', 'wall_area', 'roof_area', 'glazing_area']].corr().round(2)

,relative_area,surface_area,wall_area,roof_area,glazing_area
relative_area,1.00,-0.99,-0.20,-0.87,-0.0
surface_area,-0.99,1.00,0.20,0.88,0.0
wall_area,-0.20,0.20,1.00,-0.29,-0.0
roof_area,-0.87,0.88,-0.29,1.00,-0.0
glazing_area,-0.00,0.00,-0.00,-0.00,1.0


In [43]:
!pip install statsmodels

     |████████████████████████████████| 9.5 MB 1.8 MB/s eta 0:00:01


In [44]:
import statsmodels.api as sm

In [51]:
import statsmodels.api as sm
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           cooling_load   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     471.3
Date:                Thu, 03 Feb 2022   Prob (F-statistic):               0.00
Time:                        11:19:38   Log-Likelihood:                -1970.3
No. Observations:                 768   AIC:                             3969.
Df Residuals:                     754   BIC:                             4034.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 127.7050     18.743      6.813      0.000      90.910     164.500
height[T.low]                             -14.9935      1.281    -11.709      0.000     -17.507     -12.480
orientation[T.north]                        0.2920      0.324      0.901      0.368      -0.344       0.928
orientation[T.south]                        0.1678      0.324      0.517      0.605      -0.469       0.804
orientation[T.west]                         0.6411      0.324      1.978      0.048       0.005       1.277
glazing_area_distribution[T.no glazing]    -1.6400      0.582     -2.818      0.005      -2.782      -0.498
glazing_area_distribution[T.north]          0.3374      0.374      0.901      0.368      -0.397       1.072
glazing_area_distribution[T.south]          0.3557      0.374      0.950      0.342      -0.379       1.091
glazing_area_distribution[T.uniform]        0.5201      0.374      1.389      0.165      -0.215       1.255
glazing_area_distribution[T.west]           0.0556      0.374      0.148      0.882      -0.679       0.790
relative_area                             -70.7877     11.138     -6.355      0.000     -92.653     -48.922
surface_area                               -0.0661      0.015     -4.554      0.000      -0.095      -0.038
wall_area                                   0.0225      0.004      5.407      0.000       0.014       0.031
roof_area                                  -0.0443      0.008     -5.446      0.000      -0.060      -0.028
glazing_area                               13.2529      0.967     13.712      0.000      11.356      15.150
==============================================================================
Omnibus:                      108.750   Durbin-Watson:                   1.113
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              237.914
Skew:                           0.796   Prob(JB):                     2.18e-52
Kurtosis:                       5.214   Cond. No.                     5.22e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.67e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""